# CS524: Introduction to Optimization Lecture 25

# October 30, 2024

# Job-shop and Job-flow

Job-shop Scheduling:  There are m machines and n jobs to be processed.  Each job requires m operations, one on each machine, in a specific order (maybe given by precedence graph), but the order can be different for each job.  Each machine can only process one job at a time.  Various performance evaluation criteria.  Example: Car repair each operator (mechanic) evaluates plus schedules, gets material, etc.

Flow-shop Scheduling: A flow-shop is a shop design in which machines are arranged in series. Jobs begin processing on an initial machine, proceed through several intermediary machines, and conclude on a final machine. The series arrangement implies a linear structure to the shop. A general flow-shop is somewhat different, in that a job may skip a particular machine. For instance, although every job must proceed from 1 to N, some jobs may go from machine 1 to, say, machine 3 and then machine 4.

<h2>Example 1 : jobshop</h2>
<br>
A company has received an order for three types of wallpaper:<br>
 p1: blue background with yellow patterns<br>
 p2: green background with blue and yellow patterns<br>
 p3: yellow background with blue and green patterns<br>
Every paper type is produced as a continuous roll of paper that
passes through several machines, each printing a different color.
The order in which the papers are run through the machines
depends on the design of the paper; given in precedence set.
The processing times differ and are given in duration table.<br>
<h3>Knowing that every machine can only process one wallpaper at a time and that a paper cannot be processed by several machines simultaneously, how should the paper printing be scheduled in order to finish the order as early as possible.</h3>

In [1]:
import sys
import numpy as np

import gamspy as gp
from gamspy import Sum
# import gamspy.math as gpm

options = gp.Options(relative_optimality_gap=0)
m = gp.Container(options=options)

In [3]:
i = gp.Set(m,'i',description='paper',records=[f'p{ind+1}' for ind in range(3)])
j = gp.Alias(m,'j',i)
c = gp.Set(m,'c',description='mach',
    records=['blue','green','yellow'])
d = gp.Alias(m,'d',c)

duration = gp.Parameter(m,'duration',domain=[c,i],
    records=np.array([
        [45,   20,   12],
        [0,    10,   17],
        [10,   34,   28]]))

task = gp.Set(m,'task',domain=[c,i])
task.setRecords(duration.records)

precedence = gp.Set(m,'precedence',domain=[c,d,i], 
  records = [('blue','yellow','p1'),
      ('green','blue','p2'), ('blue','yellow','p2'),
      ('yellow','blue','p3'), ('blue','green','p3')])

# The following set will enforce paper i and j not to be done 
# on the same machine at the same time
disjoint = gp.Set(m,'disjoint',domain=[c,i,j])
disjoint[c,i,j] = gp.Number(1).where[(i.ord < j.ord) &
  task[c,i] & task[c,j]]

M = gp.Parameter(m,'M')
M[:] = Sum(task,duration[task])

# MODEL
order = gp.Variable(m,'order','binary',domain=[c,i,j],description="task(m,i) before task(m,j)")
start = gp.Variable(m,'start','positive',domain=[c,i],description="start time of task")
finish = gp.Variable(m,'finish','free',description='finish time of last task')

defstart = gp.Equation(m,'defstart', domain=[c,d,i], description='paper visits machines c, d in precedence order')
defstart[precedence[c,d,i]] = start[c,i] + duration[c,i] <= start[d,i]

deffinish = gp.Equation(m,'deffinish', domain=[c,i])
deffinish[task] = finish >= start[task] + duration[task]

# The following are either-or constraints, for each c, do paper i or paper j
disjoint1 = gp.Equation(m,'disjoint1', domain=[c,i,j], description='paper i before paper j on c')
disjoint1[disjoint[c,i,j]] = (
    start[c,i] + duration[c,i] <= start[c,j] + M*(1-order[c,i,j]) )

disjoint2 = gp.Equation(m,'disjoint2', domain=[c,i,j], description='paper j before paper i on c')
disjoint2[disjoint[c,i,j]] = (
    start[c,j] + duration[c,j] <= start[c,i] + M*order[c,i,j] )

jobshop = gp.Model(m,'jobshop',
    equations=m.getEquations(),
    problem=gp.Problem.MIP,
    sense=gp.Sense.MIN,
    objective=finish
)

jobshop.solve(options=options,solver="gurobi",output=None)

GamspyException: There was a parameter error. Check /tmp/tmpfb5e6y3l/_4361ce72-6215-4fb0-b3ef-46e3fd6d67b2.lst for more information.

In [16]:
print(f"Finish time of last task: {jobshop.objective_value}")

display(start.pivot(fill_value=''))

Finish time of last task: 97.0


,p1,p2,p3
blue,42.0,10.0,30.0
green,0.0,0.0,42.0
yellow,87.0,30.0,0.0


        c   i  level  marginal  lower  upper  scale
0    blue  p1   42.0       0.0    0.0    inf    1.0
1    blue  p2   10.0       0.0    0.0    inf    1.0
2    blue  p3   30.0       0.0    0.0    inf    1.0
3   green  p2    0.0       1.0    0.0    inf    1.0
4   green  p3   42.0       0.0    0.0    inf    1.0
5  yellow  p1   87.0       0.0    0.0    inf    1.0
6  yellow  p2   30.0       0.0    0.0    inf    1.0
7  yellow  p3    0.0      -0.0    0.0    inf    1.0


<h2>Example 2: flowshop</h2>

See separate workbook: dyeing example

## Additional Reading
This link is an article that provides a case study of job shop scheduling optimization in real-time production control to provide context outside the classroom:
https://www-sciencedirect-com.ezproxy.library.wisc.edu/science/article/pii/0166361583900064